In [28]:
import pandas as pd 

In [47]:
df = pd.read_csv('../data/raw/telco.csv')

In [57]:
df.sample(10)

,Gender,Age,Married,Dependents,Number of Dependents,Country,State,City,Population,Quarter,...,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Churn Label
5009,Female,44,No,No,0,United States,California,Huntington Beach,56517,Q3,...,No,Credit Card,49.20,103.70,0.00,0,58.56,162.26,5,No
5050,Female,42,No,No,0,United States,California,Orange,15396,Q3,...,No,Credit Card,29.90,29.90,0.00,0,0.00,29.90,4,No
1954,Male,22,No,No,0,United States,California,Kingsburg,14088,Q3,...,Yes,Bank Withdrawal,25.05,56.35,0.00,0,0.00,56.35,1,Yes
1118,Male,74,Yes,Yes,3,United States,California,Long Beach,37417,Q3,...,Yes,Bank Withdrawal,72.90,5139.65,0.00,20,3273.81,8433.46,4,No
2546,Female,57,No,No,0,United States,California,Los Angeles,47143,Q3,...,No,Mailed Check,20.65,1022.95,0.00,0,1172.60,2195.55,4,No
148,Female,65,Yes,No,0,United States,California,Lee Vining,504,Q3,...,Yes,Credit Card,94.55,3640.45,0.00,0,1980.00,5620.45,1,Yes
1191,Female,41,Yes,No,0,United States,California,La Puente,84965,Q3,...,Yes,Credit Card,44.60,80.55,0.00,0,60.38,140.93,2,Yes
2760,Female,47,No,No,0,United States,California,Vista,44692,Q3,...,Yes,Bank Withdrawal,44.95,996.85,25.84,40,0.00,1011.01,3,No
3778,Male,53,No,Yes,1,United States,California,Alhambra,30635,Q3,...,Yes,Mailed Check,21.05,21.05,0.00,0,10.14,31.19,3,No
539,Female,76,Yes,No,0,United States,California,Fountain Valley,54548,Q3,...,Yes,Bank Withdrawal,110.85,3204.40,0.00,0,1278.76,4483.16,4,No


In [66]:
df.shape


(7043, 37)

In [65]:
df.columns

Index(['Gender', 'Age', 'Married', 'Dependents', 'Number of Dependents',
       'Country', 'State', 'City', 'Population', 'Quarter',
       'Referred a Friend', 'Number of Referrals', 'Tenure in Months', 'Offer',
       'Phone Service', 'Avg Monthly Long Distance Charges', 'Multiple Lines',
       'Internet Service', 'Internet Type', 'Avg Monthly GB Download',
       'Online Security', 'Online Backup', 'Device Protection Plan',
       'Premium Tech Support', 'Unlimited Data', 'Contract',
       'Paperless Billing', 'Payment Method', 'Monthly Charge',
       'Total Charges', 'Total Refunds', 'Total Extra Data Charges',
       'Total Long Distance Charges', 'Total Revenue', 'Satisfaction Score',
       'Churn Label', 'StreamingServicesCount'],
      dtype='object')

In [51]:
df.drop(columns=["Customer ID", "Latitude", "Longitude","Zip Code","Churn Reason","Churn Category","CLTV","Churn Score","Customer Status"], inplace=True)

In [52]:
df.isnull().sum() * 100 / df.shape[0]

Gender                                0.000000
Age                                   0.000000
Under 30                              0.000000
Senior Citizen                        0.000000
Married                               0.000000
Dependents                            0.000000
Number of Dependents                  0.000000
Country                               0.000000
State                                 0.000000
City                                  0.000000
Population                            0.000000
Quarter                               0.000000
Referred a Friend                     0.000000
Number of Referrals                   0.000000
Tenure in Months                      0.000000
Offer                                55.047565
Phone Service                         0.000000
Avg Monthly Long Distance Charges     0.000000
Multiple Lines                        0.000000
Internet Service                      0.000000
Internet Type                        21.666903
Avg Monthly G

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 41 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Age                                7043 non-null   int64  
 2   Under 30                           7043 non-null   object 
 3   Senior Citizen                     7043 non-null   object 
 4   Married                            7043 non-null   object 
 5   Dependents                         7043 non-null   object 
 6   Number of Dependents               7043 non-null   int64  
 7   Country                            7043 non-null   object 
 8   State                              7043 non-null   object 
 9   City                               7043 non-null   object 
 10  Population                         7043 non-null   int64  
 11  Quarter                            7043 non-null   objec

In [54]:
df['Internet Type'] = df['Internet Type'].fillna("No Internet Service")

In [55]:
df['Offer'] = df['Offer'].fillna("No Offer")

Dropping under 30 and Senior Citizen because we can directly indetify with age column

In [56]:
df.drop(columns=["Senior Citizen", "Under 30"], inplace=True)

In [64]:
df['StreamingServicesCount'] = (
    df[['Streaming TV', 'Streaming Movies','Streaming Music']]
    .apply(lambda row: sum(row == 'Yes'), axis=1)
)

df.drop(['Streaming TV', 'Streaming Movies', 'Streaming Music'], axis=1, inplace=True)
